In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from helpers import *

In [7]:
import time

In [2]:
cursor = get_cursor()

In [3]:
from KalshiClientsBaseV2 import ExchangeClient

In [4]:
cursor.execute('select distinct(market_ticker) from ob_snapshot')
rows = cursor.fetchall()
tickers = []

for row in rows:
    tickers.append(row[0])
tickers

['INXD-23DEC28-B4687',
 'INXD-23DEC28-B4712',
 'INXD-23DEC28-B4737',
 'INXD-23DEC28-B4762',
 'INXD-23DEC28-B4787',
 'INXD-23DEC28-B4812',
 'INXD-23DEC28-B4837',
 'INXD-23DEC28-B4862',
 'INXD-23DEC28-T4675',
 'INXD-23DEC28-T4874.99',
 'NASDAQ100D-23DEC28-B16450',
 'NASDAQ100D-23DEC28-B16550',
 'NASDAQ100D-23DEC28-B16650',
 'NASDAQ100D-23DEC28-B16750',
 'NASDAQ100D-23DEC28-B16850',
 'NASDAQ100D-23DEC28-B16950',
 'NASDAQ100D-23DEC28-B17050',
 'NASDAQ100D-23DEC28-B17150',
 'NASDAQ100D-23DEC28-T16400',
 'NASDAQ100D-23DEC28-T17199.99',
 'INXD-23DEC29-B4587',
 'INXD-23DEC29-B4612',
 'INXD-23DEC29-B4637',
 'INXD-23DEC29-B4662',
 'INXD-23DEC29-B4687',
 'INXD-23DEC29-B4712',
 'INXD-23DEC29-B4737',
 'INXD-23DEC29-B4762',
 'INXD-23DEC29-B4787',
 'INXD-23DEC29-B4812',
 'INXD-23DEC29-B4837',
 'INXD-23DEC29-B4862',
 'INXD-23DEC29-B4887',
 'INXD-23DEC29-T4575',
 'INXD-23DEC29-T4899.99',
 'NASDAQ100D-23DEC29-B16150',
 'NASDAQ100D-23DEC29-B16250',
 'NASDAQ100D-23DEC29-B16350',
 'NASDAQ100D-23DEC29-B1645

In [6]:
prod_email, prod_password = get_kalshi_creds()
prod_api_base = "https://trading-api.kalshi.com/trade-api/v2" 
exchange_client = ExchangeClient(exchange_api_base=prod_api_base, email = prod_email, password = prod_password)

In [27]:
def get_interval_bounds(s):
    split = s.split(' ')
    if 'or' in s and 'below' in s:
      return [0, float(split[0].replace(',',''))]
    elif 'or' in s and 'above' in s:
      return [float(split[0].replace(',','')), np.inf]
    elif 'to' in s:
      return [float(split[0].replace(',','')), float(split[2].replace(',',''))]
    else:
      raise Exception
    

In [41]:
cursor.execute('select Date, Open, Close from spx;')
spx_rows = cursor.fetchall()
spx_d = {'Date': [], 'Open': [], 'Close': []}

cursor.execute('select Date, Open, Close from ndx;')
ndx_rows = cursor.fetchall()
ndx_d = {'Date': [], 'Open': [], 'Close': []}

for row in spx_rows:
    spx_d['Date'].append(row[0])
    spx_d['Open'].append(row[1])
    spx_d['Close'].append(row[2])
    
for row in ndx_rows:
    ndx_d['Date'].append(row[0])
    ndx_d['Open'].append(row[1])
    ndx_d['Close'].append(row[2])

spx = pd.DataFrame(data=spx_d)
ndx = pd.DataFrame(data=ndx_d)

In [56]:
abbr_to_month = {v: k for k, v in month_to_abbr.items()}
d = {'ticker': [], 'bid': [], 'ask': [], 'lb': [], 'ub': [], 'open': [], 'close': [], 'date': []}
for ticker in tickers:
    ticker_dt = ticker.split('-')[1]
    year = 2000 + int(ticker_dt[:2])
    day = int(ticker_dt[-2:])
    month = abbr_to_month[ticker_dt[2:5]]
    ts = int(time.mktime(datetime(year, month, day, 14, 30, 0).timetuple()))
    r = exchange_client.get_market_history(ticker, min_ts=ts)
    info = r['history'][0]
    first_ts = datetime.fromtimestamp(info['ts'])
    if first_ts.hour == 14:
        bb = info['yes_bid']
        ba = info['yes_ask']
        get_mark_resp = exchange_client.get_market(ticker)['market']
        subtitle = get_mark_resp['subtitle']
        ticker_date = datetime.strptime(get_mark_resp['close_time'], '%Y-%m-%dT%H:%M:%SZ').date()
        d['ticker'].append(ticker)
        d['bid'].append(bb)
        d['ask'].append(ba)
        interval = get_interval_bounds(subtitle)
        d['lb'].append(interval[0])
        d['ub'].append(interval[1])
        d['date'].append(ticker_date)
        if 'INX' in ticker:
            row = spx[spx['Date']==ticker_date]
            if row.shape[0]>0:
                d['open'].append(float(row['Open'].to_list()[0]))
                d['close'].append(float(row['Close'].to_list()[0]))
            else:
                d['open'].append(None)
                d['close'].append(None)
        else:
            row = ndx[ndx['Date']==ticker_date]
            if row.shape[0]>0:
                d['open'].append(float(row['Open'].to_list()[0]))
                d['close'].append(float(row['Close'].to_list()[0]))
            else:
                d['open'].append(None)
                d['close'].append(None)
        

df = pd.DataFrame(data=d)

In [60]:
df[(df['open'].isnull()==False) & (df['open'] >= df['lb']) & (df['open']<=df['ub'])]

,ticker,bid,ask,lb,ub,open,close,date
1,INXD-23DEC28-B4787,72,77,4775.0,4799.99,4786.44,4783.35,2023-12-28
4,NASDAQ100D-23DEC28-B16950,85,87,16900.0,16999.99,16963.52,16898.47,2023-12-28
8,INXD-23DEC29-B4787,28,32,4775.0,4799.99,4782.88,4769.83,2023-12-29
13,NASDAQ100D-23DEC29-B16950,6,7,16900.0,16999.99,16902.44,16825.93,2023-12-29
14,INXD-23DEC01-B4562,6,7,4550.0,4574.99,4559.43,4594.63,2023-12-01
18,NASDAQ100D-23DEC01-B15850,2,4,15800.0,15899.99,15892.59,15997.58,2023-12-01
25,INXD-24JAN03-B4737,50,54,4725.0,4749.99,4725.07,4704.81,2024-01-03
27,NASDAQ100D-24JAN03-B16450,61,63,16400.0,16499.99,16413.26,16368.49,2024-01-03
31,INXD-24JAN04-B4687,33,34,4675.0,4699.99,4697.42,4688.68,2024-01-04
36,NASDAQ100D-24JAN04-B16250,27,30,16200.0,16299.99,16290.78,16282.01,2024-01-04


In [55]:
float(spx[spx['Date']==date(2024, 2, 15)]['Close'].to_list()[0])

5029.73